In [1]:
import cv2
import numpy as np
import os

In [2]:
image_x,image_y=50,50

In [3]:
def create_folder(folder_name):
    if not os.path.exists(folder_name):
        os.mkdir(folder_name)

In [4]:
def store_images(g_id):
    total_pics=1200
    cap=cv2.VideoCapture(0)
    x,y,w,h=300,50,350,350
    create_folder("gestures/"+str(g_id))
    pic_no=0
    flag_start_capturing=False
    frames=0
    while True:
        ret,frame=cap.read()
        frame=cv2.flip(frame,1)
        hsv=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
        mask2=cv2.inRange(hsv,np.array([2,50,60]),np.array([25,150,255]))
        res=cv2.bitwise_and(frame,frame,mask=mask2)
        gray=cv2.cvtColor(res,cv2.COLOR_BGR2GRAY)
        median=cv2.GaussianBlur(gray,(5,5),0)
        kernel_square=np.ones((5,5),np.uint8)
        dilation=cv2.dilate(median,kernel_square,iterations=2)
        opening=cv2.morphologyEx(dilation,cv2.MORPH_CLOSE,kernel_square)
        ret,thresh=cv2.threshold(opening,30,255,cv2.THRESH_BINARY)
        thresh=thresh[y:y+h,x:x+w]
        contours=cv2.findContours(thresh.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)[1]
        if len(contours)>0:
            contour=max(contours,key=cv2.contourArea)
            if cv2.contourArea(contour)>10000 and frames>50:
                x1,y1,w1,h1=cv2.boundingRect(contour)
                pic_no+=1
                save_img=thresh[y1:y1+h1,x1:x1+w1]
                save_img=cv2.resize(save_img,(image_x,image_y))
                cv2.putText(frame,"Capturing...",(30,60),cv2.FONT_HERSHEY_TRIPLEX,2,(127,255,255))
                cv2.imwrite("gestures/"+str(g_id)+"/"+str(pic_no)+".jpg",save_img)
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
            cv2.putText(frame,str(pic_no),(30,400),cv2.FONT_HERSHEY_TRIPLEX,1.5,(127,127,255))
            cv2.imshow("Capturing gesture",frame)
            cv2.imshow("thresh",thresh)
            keypress=cv2.waitKey(1)
            if keypress==ord("c"):
                if flag_start_capturing==False:
                    flag_start_capturing=True
                else:
                    flag_start_capturing=False
                    frames=0
            if flag_start_capturing==True:
                frames+=1
            if pic_no==total_pics:
                break
                

In [ ]:
g_id=input("Enter gesture no:")
store_images(g_id)

Enter gesture no:


In [ ]:
cap=cv2.VideoCapture(0)
cap.release()
cv2.destroyAllWindows()

In [3]:
from PIL import Image
import numpy as np
import sys
import os
import csv
j=0
# default format can be changed as needed
def createFileList(myDir, format='.jpg'):
    fileList = []
    print(myDir)
    for root, dirs, files in os.walk(myDir, topdown=True):
        for name in files:
            if name.endswith(format):
                fullName = os.path.join(root, name)
                fileList.append(fullName)
    return fileList

# load the original image
mydir=["hand-emo/1","hand-emo/2","hand-emo/3","hand-emo/4","hand-emo/5","hand-emo/6",
       "hand-emo/7","hand-emo/8","hand-emo/9","hand-emo/10","hand-emo/11","hand-emo/12"]
for i in mydir:
    j=j+1
    myFileList=createFileList(i)
    for file in myFileList:
        img_file = Image.open(file)
        #img_file.show()

        # get original image parameters...
        width, height = img_file.size
        format = img_file.format
        mode = img_file.mode

        # Make image Greyscale
        img_grey = img_file.convert('L')
        #img_grey.save('result.png')
        #img_grey.show()
    
        # Save Greyscale values
        value = np.asarray(img_grey.getdata(), dtype=np.int).reshape((img_grey.size[1], img_grey.size[0]))
        for m in range(1200):
            value=np.append(value,j)
            value = value.flatten()
        with open("train_foo1.csv", 'a') as f:
            writer = csv.writer(f)
            writer.writerow(value)

hand-emo/1
hand-emo/2
hand-emo/3
hand-emo/4
hand-emo/5
hand-emo/6
hand-emo/7
hand-emo/8
hand-emo/9
hand-emo/10
hand-emo/11
hand-emo/12


In [4]:
import numpy as np
from keras.layers import Dense,Flatten,Conv2D
from keras.layers import MaxPooling2D,Dropout
from keras.utils import np_utils,print_summary
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
import pandas as pd
import keras.backend as K

Using TensorFlow backend.


In [5]:
data=pd.read_csv("train_foo.csv")
dataset=np.array(data)
np.random.shuffle(dataset)
X=dataset
Y=dataset
X=X[:,0:2500]
Y=Y[:,2500]
print(X)
print(Y)
X_train=X[0:10800,:]
X_train=X_train/255
X_test=X[10800:12001,:]
X_test=X_test/255

[[0 0 2 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]]
[ 6 10  8 ...  3  8  7]


In [6]:
Y=Y.reshape(Y.shape[0],1)
Y_train=Y[0:10800,:]
Y_train=Y_train.T
Y_test=Y[10800:12001,:]
Y_test=Y_test.T
print("number of training examples="+str(X_train.shape[0]))
print("number of test examples:"+str(X_test.shape[0]))
print("X_train shape:"+str(X_train.shape))
print("Y_train shape:"+str(Y_train.shape))
print("X_test shape:"+str(X_test.shape))
print("Y_test shape:"+str(Y_test.shape))

number of training examples=10800
number of test examples:1201
X_train shape:(10800, 2500)
Y_train shape:(1, 10800)
X_test shape:(1201, 2500)
Y_test shape:(1, 1201)


In [7]:
image_x=50
image_y=50
train_y=np_utils.to_categorical(Y_train)
test_y=np_utils.to_categorical(Y_test)
train_y=train_y.reshape(train_y.shape[1],train_y.shape[2])
test_y=test_y.reshape(test_y.shape[1],test_y.shape[2])
X_train=X_train.reshape(X_train.shape[0],50,50,1)
X_test=X_test.reshape(X_test.shape[0],50,50,1)
print("X_train shape:"+str(X_train.shape))
print("X_test shape:"+str(X_test.shape))
print("train_y shape:"+str(train_y.shape))

X_train shape:(10800, 50, 50, 1)
X_test shape:(1201, 50, 50, 1)
train_y shape:(10800, 13)


In [8]:

def keras_model(image_x,image_y):
    num_of_classes=13
    model=Sequential()
    model.add(Conv2D(32,(5,5),input_shape=(image_x,image_y,1),activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),padding="same"))
    model.add(Conv2D(64,(5,5),activation="sigmoid"))
    model.add(MaxPooling2D(pool_size=(5,5),strides=(5,5),padding="same"))
    model.add(Flatten())
    model.add(Dense(1024,activation="relu"))
    model.add(Dropout(0.6))
    model.add(Dense(num_of_classes,activation="softmax"))
    model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])
    filepath="handEmo.h5"
    checkpoint1=ModelCheckpoint(filepath,monitor="val_acc",verbose=1,save_best_only=True,mode="max")
    callbacks_list=[checkpoint1]
    return model,callbacks_list

In [9]:
model,callbacks_list=keras_model(image_x,image_y)
model.fit(X_train,train_y,validation_data=(X_test,test_y),epochs=1,batch_size=64,callbacks=callbacks_list)
scores=model.evaluate(X_test,test_y,verbose=0)
print("CNN Error:%.2f%%"%(100-scores[1]*100))
print_summary(model)
model.save("handEmo.h5")

Train on 10800 samples, validate on 1201 samples
Epoch 1/1
10800/10800 [==============================] - 68s 6ms/step - loss: 0.4920 - accuracy: 0.8679 - val_loss: 0.0032 - val_accuracy: 0.9992


C:\Users\Ankit\Anaconda3\envs\tensorflow\lib\site-packages\keras\callbacks\callbacks.py:707: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


CNN Error:0.08%
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 46, 46, 32)        832       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 19, 19, 64)        51264     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dropout_1 (Dropout)          (None, 10

In [10]:
from keras.models import load_model

In [11]:
model=load_model("handEmo.h5")

In [12]:
def get_emojis():
    emoji_folder="hand-emo/"
    emojis=[]
    for emoji in range(len(os.listdir(emoji_folder))):
        print(emoji)
        emojis.append(cv2.imread(emoji_folder+str(emoji)+".jpg",-1))
    return emojis

In [13]:
emojis=get_emojis()
cap=cv2.VideoCapture(0)
x,y,w,h=300,50,350,350

0
1
2
3
4
5
6
7
8
9
10
11


In [14]:
def keras_predict(model,image):
    processed=keras_process_image(image)
    pred_probab=model.predict(processed)[0]
    pred_class=list(pred_probab).index(max(pred_probab))
    return max(pred_probab),pred_class

In [15]:

def keras_process_image(img):
    image_x=50
    image_y=50
    img=cv2.resize(img,(image_x,image_y))
    img=np.array(img,dtype=np.float32)
    img=np.reshape(img,(-1,image_x,image_y,1))
    return img

In [16]:
def overlay(image,emoji,x,y,w,h):
    emoji=cv2.resize(emoji,(w,h))
    try:
        image[y:y+h,x:x+w]=blend_transparent(image[y:y+h,x:x+w],emoji)
    except:
        pass
    return image

In [17]:

def blend_transparent(face_img,overlay_t_img):
    overlay_img=overlay_t_img[:,:,:3]
    overlay_mask=overlay_t_img[:,:,3:]
    background_mask=255-overlay_mask
    overlay_mask=cv2.cvtColor(overlay_mask,cv2.COLOR_GRAY2BGR)
    background_mask=cv2.cvtColor(background_mask,cv2.COLOR_GRAY2BGR)
    face_part=(face_img*(1/255.0))*(backgroud_mask*(1/255.0))
    overlay_part=(overlay_img*(1/255.0))*(overlay_mask*(1/255.0))
    return np.uint8(cv.addWeighted(face_part,255.0,overlay_part,255.0,0.0))

In [18]:
while(cap.isOpened()):
    ret,img=cap.read()
    img=cv2.flip(img,1)
    hsv=cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
    mask2=cv2.inRange(hsv,np.array([2,50,60]),np.array([25,150,255]))
    res=cv2.bitwise_and(img,img,mask=mask2)
    gray=cv2.cvtColor(res,cv2.COLOR_BGR2GRAY)
    median=cv2.GaussianBlur(gray,(5,5),0)
    kernel_square=np.ones((5,5),np.uint8)
    dilation=cv2.dilate(median,kernel_square,iterations=2)
    opening=cv2.morphologyEx(dilation,cv2.MORPH_CLOSE,kernel_square)
    ret,thresh=cv2.threshold(opening,30,255,cv2.THRESH_BINARY)
    thresh=thresh[y:y+h,x:x+h]
    contours=cv2.findContours(thresh.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)[1]
    if len(contours)>0:
        contour=max(contours,key=cv2.contourArea)
        if cv2.contourArea(contour)>2500:
            x,y,w1,h1=cv2.boundingRect(contour)
            newImage=thresh[y:y+h1,x:x+w1]
            newImage=cv2.resize(newImage,(50,50))
            pred_probab,pred_class=keras_predict(model,newImage)
            print(pred_class,pred_probab)
            img=overlay(img,emojis[pred_class],400,250,90,90)
    x,y,w,h=300,50,350,350
    cv2.imshow("Frame",img)
    cv2.imshow("Contours",thresh)
    k=cv2.waitKey(10)
    if k==27:
        break
keras_predict(model,np.zeros((50,50,1),dtype=np.uint8))


3 0.5802255
3 0.9938599
3 0.7765638
3 0.38286942
4 0.25933787
4 0.31183982
4 0.28609073
2 0.26468587
2 0.28949267
2 0.3267232
2 0.31113592
2 0.3221451
11 0.31974307
1 0.27916434
1 0.4135609
1 0.4419541
1 0.34329528
11 0.35695684
1 0.2684657
2 0.27929685
11 0.36341327
1 0.4054347
1 0.31767702
2 0.32252297
1 0.3243647
1 0.406521
1 0.40442654
1 0.51697916
1 0.44860074
1 0.49877396
1 0.58772343
1 0.53423536
1 0.5680663
1 0.564932
1 0.50392
1 0.48927253
1 0.5770173
1 0.5366761
1 0.5082707
1 0.53604937
1 0.56380725
1 0.5391534
1 0.5173343
1 0.39969242
1 0.4438968
1 0.34761053
1 0.35959232
2 0.2549862
2 0.31733677
1 0.29789606
2 0.30007997
1 0.3828362
2 0.5553554
2 0.38790175
2 0.49452972
2 0.40552557
2 0.36384508
1 0.5804091
1 0.61291397
1 0.76868755
1 0.5961371
1 0.6352153
1 0.76481676
1 0.77522343
1 0.64090055
1 0.8572273
1 0.8713389
1 0.92996335
1 0.9156371
1 0.9233053
1 0.87009656
1 0.8701376
1 0.7089714
11 0.5049707
1 0.6011293
1 0.4013033
1 0.3907859
11 0.3314662
11 0.44513136
4 0.2969

IndexError: list index out of range

In [19]:
cap=cv2.VideoCapture(0)
cap.release()
cv2.destroyAllWindows()